# Search re-ranking using Gemini embeddings

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemini-api-cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [ ]:
!pip install -q -U google-generativeai

In [ ]:
!pip install -q wikipedia

Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

In [ ]:
import json
import textwrap

import google.generativeai as genai

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import numpy as np

from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Define tools

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Define the search function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-pro` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [ ]:
def wikipedia_search(search_queries: list[str]) -> list[str]:
  """Search wikipedia for each query and summarize relevant docs."""
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []
  mining_model = genai.GenerativeModel('gemini-pro')
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-pro` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content
        response = mining_model.generate_content(textwrap.dedent(f"""\
            Extract relevant information
            about user's query: {query}
            From this source:

            {page}

            Note: Do not summarize. Only Extract and return the relevant information
        """))

        urls = [url]
        if response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        try:
          text = response.text
        except ValueError:
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results


In [ ]:
example = wikipedia_search(["What are LLMs?"])

Searching for "What are LLMs?"
Related search terms: ['Large language model', 'Prompt engineering', 'Gemini (language model)']
Fetching page: "Large language model"
Information Source: https://en.wikipedia.org/wiki/Large_language_model
Fetching page: "Prompt engineering"
Information Source: https://en.wikipedia.org/wiki/Prompt_engineering
Fetching page: "Gemini (language model)"
Information Source: https://en.wikipedia.org/wiki/Gemini_(language_model)
Information Sources:
     https://en.wikipedia.org/wiki/Large_language_model
     https://en.wikipedia.org/wiki/Prompt_engineering
     https://en.wikipedia.org/wiki/Gemini_(language_model)


Here is what the search results look like:

In [ ]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

> **Definition:**
> 
> * A large language model (LLM) is an artificial neural network that can perform natural language processing tasks such as text generation, classification, and question answering.
> 
> **Training and Architecture:**
> 
> * LLMs are trained on massive text datasets using self-supervised and semi-supervised learning techniques.
> * They are typically built with transformer-based architectures and can have billions or even trillions of parameters.
> * Prompt engineering is used to optimize the performance of LLMs on specific tasks.
> 
> **Capabilities:**
> 
> * **Text Generation:** LLMs can generate human-like text, including stories, articles, and code.
> * **Language Understanding:** LLMs can understand the meaning and structure of text, including context and sentiment.
> * **Question Answering:** LLMs can answer questions based on their knowledge of the text they have been trained on.
> * **Natural Language Processing:** LLMs can perform various NLP tasks, such as sentiment analysis, machine translation, and named entity recognition.
> 
> **Multimodality:**
> 
> * LLMs can be extended to handle multiple modalities, such as images, audio, and video, by using tokenization methods.
> * This enables them to perform tasks such as image captioning, video summarization, and audio transcription.
> 
> **Properties:**
> 
> * **Scaling Laws:** The performance of LLMs scales with the size of their training data, the number of parameters, and the computational power used for training.
> * **Emergent Abilities:** LLMs exhibit emergent abilities that are not explicitly programmed into them, such as in-context learning and solving complex reasoning problems.
> * **Interpretation:** Researchers are exploring methods to interpret the inner workings of LLMs and understand how they process and generate language.
> 
> **Wider Impact:**
> 
> * LLMs have potential applications in various industries, including healthcare, education, and customer service.
> * They also raise concerns about copyright infringement, algorithmic bias, and the spread of misinformation.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Large_language_model

> **What are LLMs?**
> 
> Large Language Models (LLMs) are a type of generative AI model that can be trained on large amounts of text data to understand and generate human-like language. They are used in a variety of applications, including:
> 
> * **Text generation:** LLMs can generate text that is indistinguishable from human-written text. This can be used for a variety of purposes, such as writing articles, stories, and marketing copy.
> * **Machine translation:** LLMs can be used to translate text from one language to another.
> * **Question answering:** LLMs can be used to answer questions by searching through large amounts of text data.
> * **Summarization:** LLMs can be used to summarize large amounts of text into shorter, more manageable summaries.
> 
> LLMs are still under development, but they have the potential to revolutionize the way we interact with computers. They could make it possible for us to communicate with computers in a more natural way, and they could help us to access and understand information more easily.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Prompt_engineering

> - Large language models (LLMs) are advanced AI models that can process and generate human-like text.
> - LLMs are trained on massive datasets of text and code, which allows them to understand and respond to a wide range of natural language queries.
> - LLMs are used in a variety of applications, including search engines, chatbots, and machine translation.
> - Gemini is a family of LLMs developed by Google DeepMind.
> - Gemini comprises three models: Gemini Ultra, Gemini Pro, and Gemini Nano.
> - Gemini Ultra is designed for "highly complex tasks", Gemini Pro is designed for "a wide range of tasks", and Gemini Nano is designed for "on-device tasks".
> - Gemini was launched on December 6, 2023.
> - Gemini is available through Google Cloud's Vertex AI service.
> - Gemini is trained on and powered by Google's Tensor Processing Units (TPUs).
> - Gemini has outperformed GPT-4, Anthropic's Claude 2, Inflection AI's Inflection-2, Meta's LLaMA 2, and xAI's Grok 1 on a variety of industry benchmarks.
> - Gemini Pro is made available to Google Cloud customers on AI Studio and Vertex AI on December 13, while Gemini Nano will be made available to Android developers as well.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Gemini_(language_model)

### Pass the tools to the model

If you pass a list of functions to the `GenerativeModel`'s `tools` argument,
it will extract a schema from the function's signature and type hints, and then pass schema along to the API calls. In response the model may return a `FunctionCall` object asking to call the function.

Note: This approach only handles annotations of `AllowedTypes = int | float | str | dict | list['AllowedTypes']`

The `GenerativeModel` will keep a reference to the function inself, so that it _can_ execute the function locally later.

In [ ]:
model = genai.GenerativeModel(
    'gemini-pro',
    tools=[wikipedia_search],
    generation_config={'temperature': 0.6})

## Generate supporting search queries

In order to have multiple supporting search queries to the user's original query, you will ask the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [ ]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

In order to yield creative and a more random variety of questions, you will set the model's temperature parameter to a value higher. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

## Enable automatic function calling and call the API

Now start a new chat with `enable_automatic_function_calling=True`. With it enabled, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [ ]:
model = genai.GenerativeModel(
    'gemini-pro', tools=[wikipedia_search], generation_config={'temperature': 0.6})

chat = model.start_chat(enable_automatic_function_calling=True)

query = "Explain how deep-sea life survives."

res = chat.send_message(instructions.format(query=query))

Searching for "How do deep-sea creatures survive in the deep sea?"
Related search terms: ['Deep-sea fish', 'Deep-sea community', 'Deep sea']
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Deep sea"
Information Source: https://en.wikipedia.org/wiki/Deep_sea
Searching for "What adaptations have deep-sea creatures developed to survive in the deep sea?"
Related search terms: ['Deep-sea fish', 'Deep-sea community', 'Deep sea']
Searching for "What is the pressure at the bottom of the ocean?"
Related search terms: ['DSV Limiting Factor', 'Titan submersible implosion', 'Seabed']
Fetching page: "DSV Limiting Factor"
Information Source: https://en.wikipedia.org/wiki/DSV_Limiting_Factor
Fetching page: "Titan submersible implosion"
Information Source: https://en.wikipedia.org/wiki/Titan_submersible_implosion
Fetching page: "Seabed"

In [ ]:
to_markdown(res.text)

> Deep-sea life survives in the extreme conditions of the deep sea by developing various adaptations, including:
> 
> * **Pressure Resistance:** Deep-sea organisms have evolved to withstand the crushing pressure of the deep sea, which can reach up to 1,000 times the pressure at sea level. Their bodies have a high internal pressure to balance the external pressure and prevent them from being crushed.
> 
> * **Low Light Adaptation:** Many deep-sea fish have evolved large and sensitive eyes to maximize light absorption in the dark depths. Some species have multiple Rh1 opsin genes, which enhance their ability to see in low light conditions.
> 
> * **Bioluminescence:** Many deep-sea creatures produce their own light through bioluminescent organs to attract prey, communicate, or camouflage themselves.
> 
> * **Neutral Buoyancy:** Deep-sea fish often have low tissue density and reduced skeletal weight to maintain neutral buoyancy in the water column. Some species have swim bladders to help them adjust their buoyancy.
> 
> * **Sensing Adaptations:** To navigate and locate prey in the dark, deep-sea creatures rely on highly sensitive senses such as touch, smell, and electroreception.
> 
> * **Slow Metabolism and Energy Conservation:** Deep-sea organisms have adapted to survive on limited food sources. They have slower metabolisms and specialized feeding behaviors, such as lying in wait for prey or using lures to attract it.

Check for additional citations:

In [ ]:
res.candidates[0].citation_metadata or 'No citations found'

'No citations found'

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [ ]:
for content in chat.history:
  part = content.parts[0]

  print(f'{content.role} -> ', end='')
  print(json.dumps(type(part).to_dict(part), indent=2))
  print('---' * 20)


user -> {
  "text": "You have access to the Wikipedia API which you will be using\nto answer a user's query. Your job is to generate a list of search queries which\nmight answer a user's question. Be creative by using various key-phrases from\nthe user's query. To generate variety of queries, ask questions which are\nrelated to  the user's query that might help to find the answer. The more\nqueries you generate the better are the odds of you finding the correct answer.\nHere is an example:\n\nuser: Tell me about Cricket World cup 2023 winners.\n\nfunction_call: wikipedia_search(['What is the name of the team that\nwon the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup\n2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What\nwas the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',\n'Who lifted the Cricket World Cup 2023 trophy?'])\n\nThe search function will return a list of article summaries, use these to\nans

In the chat history you can see all 4 steps:

1. The user sent the query.
2. The model replied with a `glm.FunctionCall` calling the `wikipedia_search` with a number of relevant searches.
3. Because you set `enable_automatic_function_calling=True` when creating the `genai.ChatSession`, it  executed the search function and returned the list of article summaries to the model.
4. Folliwing the instructions in the prompt, the model generated a final answer based on those summaries.


## [Optional] Manually execute the function call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [ ]:
import google.ai.generativelanguage as glm # for lower level code

In [ ]:
chat = model.start_chat()

In [ ]:
result = chat.send_message(instructions.format(query=query))

Initially the model returns a FunctionCall:

In [ ]:
fc = result.candidates[0].content.parts[0].function_call
fc = type(fc).to_dict(fc)
print(json.dumps(fc, indent=2))

{
  "name": "wikipedia_search",
  "args": {
    "search_queries": [
      "How do deep-sea creatures survive the immense pressure of the deep sea?",
      "What adaptations have deep-sea creatures evolved to withstand the extreme conditions of their environment?",
      "How do deep-sea creatures generate their own light?",
      "What is the role of bioluminescence in the deep sea?",
      "How do deep-sea creatures find food in the darkness of the deep sea?",
      "What are the unique challenges faced by deep-sea creatures?",
      "How do deep-sea creatures reproduce in the extreme conditions of their environment?"
    ]
  }
}


In [ ]:
fc['name']

'wikipedia_search'

Call the function with generated arguments to get the results.

In [ ]:
summaries = wikipedia_search(**fc['args'])

Searching for "How do deep-sea creatures survive the immense pressure of the deep sea?"
Related search terms: ['Deep-sea community', 'Sea', 'Ocean']
Fetching page: "Deep-sea community"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_community
Fetching page: "Sea"
Information Source: https://en.wikipedia.org/wiki/Sea
Fetching page: "Ocean"
Information Source: https://en.wikipedia.org/wiki/Ocean
Searching for "What adaptations have deep-sea creatures evolved to withstand the extreme conditions of their environment?"
Related search terms: ['Deep-sea fish', 'Astrobiology', 'Marine life']
Fetching page: "Deep-sea fish"
Information Source: https://en.wikipedia.org/wiki/Deep-sea_fish
Fetching page: "Astrobiology"
Information Source: https://en.wikipedia.org/wiki/Astrobiology
Fetching page: "Marine life"
Information Source: https://en.wikipedia.org/wiki/Marine_life
Searching for "How do deep-sea creatures generate their own light?"
Related search terms: ['Bioluminescence', 'The Deep

Now send the `FunctionResult` to the model.

In [ ]:
response = chat.send_message(
    glm.Content(
      parts=[glm.Part(
          function_response = glm.FunctionResponse(
            name='wikipedia_search',
            response={'result': summaries}
          )
      )]
    )
)

to_markdown(response.text)

> Deep-sea creatures have evolved unique adaptations to survive the immense pressure of the deep sea. These adaptations include having flexible bodies, using hydrostatic pressure to their advantage, and having specialized proteins that protect their cells from damage.
> 
> Deep-sea creatures also have a number of adaptations that help them find food in the darkness of the deep sea. These adaptations include having large eyes that are sensitive to low levels of light, and having long, sensitive tentacles that can detect prey.
> 
> Some deep-sea creatures also have bioluminescent organs that they use to attract prey or communicate with each other.

## Re-ranking the search results

Helper function to embed the content:

In [ ]:
def get_embeddings(content: list[str]) -> np.ndarray:
  embeddings = genai.embed_content('models/embedding-001', content, 'SEMANTIC_SIMILARITY')
  embds = embeddings.get('embedding', None)
  embds = np.array(embds).reshape(len(embds), -1)
  return embds

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and hence their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [ ]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [ ]:
search_res = get_embeddings(summaries)
embedded_query = get_embeddings([query])

Calculate similarity score:

In [ ]:
sim_value = dot_product(search_res, embedded_query)

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [ ]:
print(summaries[np.argmax(sim_value)])

Deep-sea creatures survive the immense pressure of the deep sea by evolving adaptations that allow them to withstand the extreme conditions. These adaptations include having flexible bodies, using hydrostatic pressure to their advantage, and having specialized proteins that protect their cells from damage.

Based on:
  https://en.wikipedia.org/wiki/Sea


### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-pro`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [ ]:
hypothetical_ans_model = genai.GenerativeModel('gemini-pro')
res = hypothetical_ans_model.generate_content(f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}""")

to_markdown(res.text)

> In the abyssal depths of the ocean, where sunlight fades into eternal darkness and pressure increases with each descending fathom, life adapts to the extreme with remarkable resilience.
> 
> At the core of this survival lies a unique set of adaptations that optimize energy conservation and maximize resource utilization. Organisms here harness the power of [unique chemical reactions] and [specialized organs] to extract sustenance from the meager organic matter that drifts down from the sunlit surface.
> 
> The frigid temperatures and high-pressure environment necessitate a [slowed metabolism] that dramatically reduces energy expenditure. Organisms have evolved [flexible structures] that withstand the crushing forces, and [specialized enzymes] that function optimally in the cold and darkness.
> 
> Many deep-sea creatures have developed [bioluminescent capabilities], creating their own light to attract mates or lure prey. These glowing displays illuminate the otherwise pitch-black surroundings, facilitating communication and survival.
> 
> Moreover, deep-sea life exhibits [symbiotic relationships] with other organisms. Some species form partnerships with [bacteria that provide nourishment], while others establish alliances with [larger predators that offer protection]. These codependencies foster survival in an environment where resources are scarce.
> 
> With their exceptional adaptations, deep-sea creatures defy the odds, thriving in an environment that would crush most other life forms. Their resilience and ingenuity serve as a testament to the boundless wonders that lie hidden in the unfathomable depths of our oceans.

Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [ ]:
hypothetical_ans = get_embeddings([res.text])

Calculate similarity scores to rank the search results

In [ ]:
sim_value = dot_product(search_res, hypothetical_ans)

In [ ]:
sim_value

array([[0.74144238],
       [0.81955475],
       [0.83515347],
       [0.79930045],
       [0.78609111],
       [0.83174707],
       [0.77690633],
       [0.69279512],
       [0.69031457],
       [0.73286281],
       [0.73241528],
       [0.74342376],
       [0.79973473],
       [0.67441962],
       [0.70701801]])

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [ ]:
to_markdown(summaries[np.argmax(sim_value)])

> Deep-sea creatures can endure the immense pressure in the deep sea due to a variety of adaptations. These adaptations include:
> 
> * **Having a body that is made up mostly of water.** This reduces the amount of pressure that the creature's body experiences.
> * **Having a thick, tough skin that is resistant to pressure.** This helps to protect the creature's body from the crushing pressure of the water.
> * **Having a skeleton that is made of a material that is stronger than bone.** This helps to support the creature's body and prevent it from collapsing under pressure.
> * **Having a circulatory system that is adapted to high pressure.** This helps to ensure that the creature can get the oxygen and nutrients it needs to survive.
> * **Having a metabolism that is slowed down.** This helps to conserve energy and prevent the creature from overheating.
> 
> These adaptations allow deep-sea creatures to survive in the extreme conditions of the deep sea.
> 
> Based on:
>   https://en.wikipedia.org/wiki/Ocean

You have now created a search re-ranking engine using embeddings!

## Next steps

I hope you found this example helpful! Check out more examples in the [Gemini Guide](https://github.com/google-gemini/gemini-guide/) to learn more.